In [4]:
%matplotlib inline
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from typing import Dict, Any, Optional
import plotly.express as px
import ast

plt.style.use("../plt_plot_style.mplstyle")
f_name = "../project.csv"
df = pd.read_csv(f_name)
df.head()

,Unnamed: 0,summary,config,name
0,0,"{'z-linear_total-active-neurons': 10, 'z-layer...","{'rigl': {'alpha': 0.3, 'delta': 100, 'use_t_e...",resnet18_cifar10_cedar_speed_test
1,1,{'z-layer4.0.shortcut.0-Max-Inactive_Weight': ...,"{'rigl': {'alpha': 0.3, 'delta': 100, 'use_t_e...",resnet18_cifar10_dense_alloc-0.1_const_fan_in-...
2,2,"{'z-layer2.1.conv2_total-active-neurons': 127,...","{'rigl': {'alpha': 0.3, 'delta': 100, 'use_t_e...",resnet18_cifar10_dense_alloc-0.1_const_fan_in-...
3,3,"{'z-linear_total-active-neurons': 10, 'z-conv1...","{'rigl': {'alpha': 0.3, 'delta': 100, 'use_t_e...",resnet18_cifar10_dense_alloc-0.1_const_fan_in-...
4,4,{'z-layer3.1.conv2-Max-Grad-Per-Filter': {'_ty...,"{'rigl': {'alpha': 0.3, 'delta': 100, 'use_t_e...",resnet18_cifar10_dense_alloc-0.1_const_fan_in-...


In [5]:
df["config"] = df["config"].apply(lambda x: pd.json_normalize(ast.literal_eval(x)))

In [78]:
pd.json_normalize(ast.literal_eval(df["config"].iloc[0]))

,rigl.alpha,rigl.delta,rigl.use_t_end,rigl.static_topo,rigl.const_fan_in,rigl.erk_power_scale,rigl.init_method_str,rigl.static_ablation,rigl.dense_allocation,rigl.dynamic_ablation,...,training.log_interval,training.weight_decay,training.warm_up_steps,training.label_smoothing,training.test_batch_size,training.simulated_batch_size,experiment.name,experiment.run_id,experiment.comment,experiment.resume_from_checkpoint
0,0.3,100,True,0,True,1,grad_flow_init,False,0.01,True,...,1000,0.0005,0,0,1000,None,resnet18_cifar10_cedar_speed_test,None,cedar_speed_test,False


In [ ]:
def parse_wandb_results(project_results_csv: str) -> pd.DataFrame:
    df = pd.read_csv(project_results_csv)
    df["summary"]

In [4]:
_NAN_TOKEN="N/A"

def parse_summary(dict_summary: Dict[str, Any]):
    _TARGET_KEYS=[
        "accuracy",
        "best_accuracy",
        "ITOP Rate",
        "_TOTAL_PERCENTAGE_ACTIVE_NEURONS",
        
    ]
    selected_values = {}
    summary = ast.literal_eval(dict_summary)
    for target in _TARGET_KEYS:
        if target in summary:
            selected_values[target] = summary[target]
        else:
            selected_values[target] = _NAN_TOKEN
    return selected_values


    
parse_summary(dict_summary=df.iloc[0]["summary"])


def parse_results(df: pd.DataFrame):
    df["summary"].apply(parse_summary)
    df["config"].apply(parse_str_dict)
    return df


test_df = parse_results(df)
test_df
    
    
        

ValueError: Duplicated key=rigl.const_fan_in
Dict so far ={'rigl.alpha': 0.3, 'rigl.delta': 100, 'rigl.static_topo': 0, 'rigl.const_fan_in': False, 'rigl.erk_power_scale': 1, 'rigl.dense_allocation': 0.1, 'rigl.grad_accumulation_n': 1, 'rigl.sparsity_distribution': 'erk', 'model': 'resnet18', 'paths.logs': '/home/condensed-sparsity/logs', 'paths.artifacts': '/home/condensed-sparsity/artifacts', 'paths.data_folder': '/home/condensed-sparsity/data', 'wandb.entity': 'condensed-sparsity', 'wandb.project': 'condensed-rigl', 'wandb.start_method': 'thread', 'compute.no_cuda': False, 'cuda_kwargs.shuffle': True, 'cuda_kwargs.pin_memory': True, 'cuda_kwargs.num_workers': 1, 'dataset': 'cifar10', 'training.lr': 0.1, 'training.seed': 4398, 'training.gamma': 0.2, 'training.epochs': 250, 'training.dry_run': False, 'training.init_lr': 1e-06, 'training.momentum': 0.9, 'training.optimizer': 'sgd', 'training.scheduler': 'step_lr', 'training.step_size': 30000, 'training.batch_size': 128, 'training.save_

In [2]:
conf = df.iloc[0]["config"]

def parse_str_dict(str_dict_rep: str) -> Dict[str, Any]:
    d = ast.literal_eval(str_dict_rep)
    d = flatten_dict(d)
    return d

def flatten_dict(d: Dict[str, Any], flat_dict: Optional[Dict[str, Any]]=None, key_prefix: Optional[str] = ""):
    if flat_dict is None:
        flat_dict = {}
    for k,v in d.items():
        if isinstance(v, dict):
            flat_dict = flatten_dict(v, flat_dict, key_prefix=f"{k}.")
            continue
        else:
            if f"{key_prefix}{k}" in flat_dict:
                if flat_dict[f"{key_prefix}{k}"] != v:
                    raise ValueError(
                        f"Duplicated key={key_prefix}{k}\n"
                        f"Dict so far ={flat_dict}\n"
                        f"I got two values: {flat_dict[f'{key_prefix}{k}']} & "
                        f"{v}")
                continue
            else:
                flat_dict[f"{key_prefix}{k}"]=v
    return flat_dict
        
flat_dict=parse_str_dict(conf)
flat_dict

{'rigl.alpha': 0.3,
 'rigl.delta': 100,
 'rigl.use_t_end': True,
 'rigl.static_topo': 0,
 'rigl.const_fan_in': True,
 'rigl.erk_power_scale': 1,
 'rigl.init_method_str': 'grad_flow_init',
 'rigl.static_ablation': False,
 'rigl.dense_allocation': 0.01,
 'rigl.dynamic_ablation': True,
 'rigl.grad_accumulation_n': 1,
 'rigl.ignore_linear_layers': False,
 'rigl.sparsity_distribution': 'erk',
 'rigl.filter_ablation_threshold': None,
 'rigl.use_sparse_initialization': True,
 'rigl.min_salient_weights_per_neuron': 0.01,
 'rigl.use_sparse_const_fan_in_for_ablation': True,
 'model.name': 'resnet18',
 'model.activation_function': 'relu',
 'paths.base': '/home/mklasby/projects/def-yani/mklasby/condensed-sparsity',
 'paths.logs': '/localscratch/mklasby.52601981.0/output/logs',
 'paths.graphs': '/localscratch/mklasby.52601981.0/output/graphs',
 'paths.artifacts': '/localscratch/mklasby.52601981.0/output/artifacts',
 'paths.checkpoints': '/localscratch/mklasby.52601981.0/output/checkpoints',
 'paths

In [17]:
conf = dict(df.iloc[0]["config"])
conf.keys()


ValueError: dictionary update sequence element #0 has length 1; 2 is required